# Demo notebook for article recommandation

The goal of this notebook is to show a typical recommandation workflow, as it will be performed through the chatbot.

Here we only demonstrate the use of the pretrained models and explore some ways of personnalizing the recommandation.


### Contents

__Preliminaries__
* a. Package Imports
* b. Data and models

__1. Geo & Topic prediction__

* Prediction of input article's geographic zone and topic

__2. Basic reco based on article body similarity__

* a. Most similar article
    
* b. Random articles from the same cluster

__3. Similarity-based reco with filter on geo and topic__

__4. How to use the title__

__5. Entity-based recommandation__  




### Preliminaries [ to do : clean imports]

A. Packages

In [1]:
## Classic packages ##

import os 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import random 
random.seed(a=2905) # set random seed 
from random import choices
import pickle
import re
from scipy import stats
from time import time

## NLP packages ##

import gensim
from gensim import corpora

from wordcloud import WordCloud

import spacy
try: 
    print("fr_core_news_sm loaded")
    nlp = spacy.load("fr_core_news_sm") # load pre-trained models for French
except:
    print("fr loaded")
    nlp=spacy.load('fr') # fr calls fr_core_news_sm 
from spacy.lang.fr import French

import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer # not adapted to French?
from nltk.stem.snowball import FrenchStemmer # already something 

## ML with sklearn ##

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer 

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
import sklearn.cluster
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score, cross_validate

from sklearn.base import TransformerMixin
from sklearn.compose import make_column_selector

from sklearn.metrics import classification_report
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity

fr_core_news_sm loaded


B. Data and models

* Dataset with Past articles = our database for article recommandation

* User input = some recent article manually picked on the Internet

We need: 

* tfidf model to transform the article body --> saved pipeline 
* models for geo and topic classification --> saved pipeline
* cosine distance for basic comparison --> python
* kmeans model for clustering --> still to save


In [23]:
# Kmeans and corresponding tfidf

"""
with open('tfidf_vectorizer_base', 'rb') as file:
    tfidf_vectorizer_base = pickle.load(file)

with open('tfidf_vectorizer_vocab', 'rb') as file:
    tfidf_vectorizer_vocab = pickle.load(file)
"""

with open('tfidf_vectorizer_vocab_LOC', 'rb') as file:
    tfidf_vectorizer_vocab_LOC = pickle.load(file)

with open('kmeans_vocab_LOC', 'rb') as file:
    kmeans_vocab_LOC = pickle.load(file)


In [3]:
# Load pipelines from saved pickle files


with open('best_topic_lr_basic_vocab', 'rb') as file:
    best_topic_lr_basic_vocab = pickle.load(file)

with open('best_topic_lr_basic_vocab_l2', 'rb') as file:
    best_topic_lr_basic_vocab_l2 = pickle.load(file)
    
with open('best_topic_rf_basic_vocab', 'rb') as file:
    best_topic_rf_basic_vocab = pickle.load(file)

with open('best_geo_rf_entity_vocab', 'rb') as file:
    best_geo_rf_entity_vocab = pickle.load(file)
    


In [4]:
# topic codes

with open('geo_code_dic', 'rb') as file:
    geo_code_dic = pickle.load(file)
    
with open('topic_code_dic', 'rb') as file:
    topic_code_dic = pickle.load(file)
    
print(geo_code_dic)
print(topic_code_dic)

{0: 'afr', 1: 'as', 2: 'eu', 3: 'fr', 4: 'lat', 5: 'me', 6: 'spa', 7: 'usa', 8: 'wo'}
{0: 'cu', 1: 'eco', 2: 'ju', 3: 'mi', 4: 'po', 5: 'sc', 6: 'so', 7: 'spo'}


In [5]:
labeled_df=pd.read_csv("labeled_articles_clean.csv")
labeled_df.head()

,Unnamed: 0,Unnamed: 0.1,year,title,text,url,geo,topic,clean_text,clean_title,pre_title,pre_text,geo_code,topic_code
0,0,0,1988,Tintin dans l'espace,Trois semaines à bord de la station soviétique...,https://www.lexpress.fr/informations/tintin-da...,fr,sc,semain bord station soviet jean-loup chrétien ...,tintin espac,tintin espac,semain bord station soviet jean-loup chrétien ...,3,5
1,1,1,1988,Le faux suicide de Robert Boulin,1979 : son corps est découvert en forêt de Ram...,https://www.lexpress.fr/actualite/politique/le...,fr,ju,corp découvert forêt rambouillet fauss pist né...,faux suicid robert boulin,faux suicid robert boulin,corp découvert forêt rambouillet fauss pist né...,3,2
2,2,2,1988,Des pierres contre les certitudes,"Rideaux de fer baissés, silhouettes furtives, ...",https://www.lexpress.fr/actualite/monde/proche...,me,po,rideau baiss silhouet furtiv jérusalem arab to...,pierr contr certitud,pierr contr certitud,rideau baiss silhouet furtiv jérusalem arab to...,5,4
3,3,3,1988,"Otages: soudain, mercredi soir...",""" Je lui ai dit: ""Ça suffit"", et j'ai raccroch...",https://www.lexpress.fr/informations/otages-so...,me,ju,suff raccroch trop souvent échaud trop souvent...,otag soudain mercred soir,otag soudain mercred soir,suff raccroch trop souvent échaud trop souvent...,5,2
4,4,4,1988,Les secrets de la planète rouge,"S'il existe, dans le système solaire, un seul ...",https://www.lexpress.fr/actualite/sciences/les...,spa,sc,exist system solair seul endroit exobiolog dés...,secret planet roug,secret planet roug,exist system solair seul endroit exobiolog dés...,6,5


In [6]:
sample_df=pd.read_excel("sample_articles.xlsx")
sample_df.head(20)

,title,text,url,year,geo,topic,geo_code,topic_code
0,"« L’Ickabog », de J. K. Rowling : le triste ro...",C’est l’histoire d’un conte de fées qui se tra...,https://www.lemonde.fr/livres/article/2020/12/...,2020,eu,cu,2,0
1,Covid-19 en France : Jean Castex expose la str...,Le premier ministre doit présenter la stratégi...,https://www.lemonde.fr/planete/article/2020/12...,2020,fr,po,3,4
2,Claude Guéant mis en examen pour « association...,L’ex-ministre de l’intérieur était déjà mis en...,https://www.lemonde.fr/societe/article/2020/12...,2020,fr,ju,3,2
3,Pédocriminalité : quinze ans de réclusion requ...,L’homme de 70 ans comparait à huis clos pour r...,https://www.lemonde.fr/societe/article/2020/12...,2020,fr,ju,3,2
4,Le prix littéraire Interallié décerné à Irène ...,"L’autrice, qui raconte les suites du meurtre d...",https://www.lemonde.fr/culture/article/2020/12...,2020,fr,cu,3,0
5,Les championnats de foot reprennent au Maroc e...,Affaiblis économiquement par l’interruption de...,https://www.lemonde.fr/afrique/article/2020/12...,2020,afr,spo,0,7
6,Le scandale des employées de maison,"Originaire d’Asie du Sud-Est et, plus récemmen...",https://www.monde-diplomatique.fr/mav/174/LAUR...,2020,me,so,5,6
7,Mosaïque ethnique dans une zone de tensions,Selon le chef de la Commission soudanaise des ...,https://www.monde-diplomatique.fr/cartes/mosai...,2020,afr,po,0,4
8,Une population réunie dans la rue contre l’oli...,L’ex-première ministre moldave Maïa Sandu a ba...,https://www.monde-diplomatique.fr/2016/10/BEUR...,2020,eu,po,2,4
9,"Outre-Rhin aussi, médecins et infirmiers dénon...",Monsieur Axel Hopfmann a longtemps été infirmi...,https://www.monde-diplomatique.fr/2020/09/KNAE...,2020,eu,so,2,6


## 1. Geo and Topic prediction 

__Preprocessing of texts__ 

In [7]:
print(labeled_df.clean_text[0])

semain bord station soviet jean-loup chrétien premi ouest-européen sort espac orgueil survol human mont cerveau certain sien général étoil torch tranquill douzain escargot bourgogn pénich restaur moskov relev gourmandis gorg bordeau potel chabot apprêt quelqu heur transperc voût célest rejoindr station soviet pétard ton allum sous fess plus stress ordinair class jean-loup chrétien sauf incident derni minut auquel remplac doublur michel tognin premi spationaut ouest-européen effectu mission longu dur semain march vid spatial voilà vibr fibr cocardi revigor moral armé trop souvent dérout quinquagénair champion espac affich san moll machin fait bav aucun raison ménag beau bigeard petit jeunot piétinent port centrifug caisson altitud tabouret tourn autr supplic initiat devront patient rejoindr confrer élus elus chanceux rien plus temp grison trait fin modest naturel chrétien jou antihéros ceux fantasment looping chuck yeag glorieux retour façon john glen gagarin changent chaîn tromp film p

In [12]:
# preprocessing for the sample articles
my_fr_stop = {'celui-ci', 'de', 'serait', 'fussions', 'aux', 'seriez', 'c', 'notre', 'une', 'seraient', 'avons', 'eûmes', 'ayantes', 'sur', 'alors', 'ayons', 'ait', 'du', 'ils', 'serai', 'fussent', 'vingt', 'avais', 'eus', 'eurent', 'm', 'ne', 'fûtes', 'sept', 'neuf', 'fusses', 'la', 'eussiez', 'vos', 'eût', 'te', 'huit', 'aura', 'j', 'étante', 'eut', 'ont', 'elle', 'leur', 'avec', 'ni', 'mais', 'quatre', 'aviez', 'sera', 'tes', 'ayants', 'à', 'soient', 'votre', 'ou', 'y', 'et', 'avions', 'je', 'fusse', 'fût', 'fut', 'd', 'ma', 'ce', 'il', 'étaient', 'par', 'ton', 'donc', 'soyons', 'les', 'sommes', 'ayante', 'dix', 'treize', 'comme', 'aient', 'seront', 'aie', 'eûtes', 'que', 'étais', 'ayez', 'auriez', 'seras', 'eux', 'n', 'vous', 'eussions', 'fûmes', 'ayant', 'deux', 'étiez', 'aurait', 'ai', 'auraient', 'cinquante', 'étions', 'cette', 'avoir', 't', 'nous', 'des', 'cent', 'toi', 'quinze', 'qui', 'sont', 'or', 'suis', 'serons', 'eue', 'au', 'cinq', 'seize', 'quarante', 'me', 'où', 'pour', 'car', 'étant', 'depuis', 'était', 'l', 's', 'même', 'es', 'avait', 'sois', 'le', 'avaient', 'son', 'étés', 'soixante', 'onze', 'être', 'avez', 'aurais', 'auras', 'eusses', 'nos', 'étées', 'étantes', 'furent', 'ainsi', 'ses', 'six', 'autre', 'eues', 'été', 'est', 'aies', 'se', 'celui-là', 'faire', 'soit', 'aurez', 'pas', 'en', 'moi', 'sa', 'on', 'auront', 'êtes', 'trente', 'quoi', 'quatorze', 'douze', 'étée', 'mes', 'serions', 'fussiez', 'après', 'cet', 'mon', 'ta', 'trois', 'soyez', 'étants', 'dans', 'qu', 'as', 'celui', 'eu', 'quand', 'serez', 'lui', 'aurons', 'eusse', 'ces', 'tu', 'eussent', 'fus', 'avant', 'aurions', 'aurai', 'serais', 'un'}

parser = French()
stemmer = FrenchStemmer()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif '@' in str(token):
            lda_tokens+=str(token).split('@')
        else:
            lda_tokens.append(token.lower_)
    return [t for t in lda_tokens if len(str(t))>0]

def prepare_text_stem(text):
    """
    Input:
    ------
    text: string, raw text
    
    Output:
    ------
    tokens: list of string, tokenized, filtered and lemmatized words from the input text
    """
    tokens = tokenize(text) # split and lower case
    tokens=[re.sub(r'\b\d+\b', '', token).strip(' ') for token in tokens] # get rid of digits
    tokens = [token for token in tokens if len(token) > 3] # arbitrary length, +get rid of empty strings
    tokens = [token for token in tokens if token not in my_fr_stop] # stopwords
    #print("Remaining tokens : ", tokens)
    tokens = [stemmer.stem(token) for token in tokens] # obtain lemmas
    return tokens  

title_tokens = []
text_tokens = []

## Apply on titles ##

for t in sample_df.title:
    tokens = prepare_text_stem(t)
    title_tokens.append(tokens)
        
## Apply on titles ##

for t in sample_df.text:
    tokens = prepare_text_stem(t)
    text_tokens.append(tokens)

print(sample_df.title[:1])        
print(title_tokens[:1])
print(sample_df.text[:1][:20])  
print(text_tokens[0][:10])

# save the preprocessed text
sample_df["clean_text"]=[' '.join(text_tokens[i]) for i in range(len(text_tokens))]
sample_df["clean_title"]=[' '.join(title_tokens[i]) for i in range(len(title_tokens))]

0    « L’Ickabog », de J. K. Rowling : le triste ro...
Name: title, dtype: object
[['ickabog', 'rowling', 'trist', 'royaum', 'marcheur']]
0    C’est l’histoire d’un conte de fées qui se tra...
Name: text, dtype: object
['histoir', 'cont', 'fé', 'transform', 'cont', 'fous', 'jusqu', 'cornucopi', 'royaum', 'auss']


In [13]:
sample_df.head()

,title,text,url,year,geo,topic,geo_code,topic_code,clean_text,clean_title
0,"« L’Ickabog », de J. K. Rowling : le triste ro...",C’est l’histoire d’un conte de fées qui se tra...,https://www.lemonde.fr/livres/article/2020/12/...,2020,eu,cu,2,0,histoir cont fé transform cont fous jusqu corn...,ickabog rowling trist royaum marcheur
1,Covid-19 en France : Jean Castex expose la str...,Le premier ministre doit présenter la stratégi...,https://www.lemonde.fr/planete/article/2020/12...,2020,fr,po,3,4,premi ministr doit présent strateg gouvern vac...,covid- franc jean castex expos strateg gouvern...
2,Claude Guéant mis en examen pour « association...,L’ex-ministre de l’intérieur était déjà mis en...,https://www.lemonde.fr/societe/article/2020/12...,2020,fr,ju,3,2,ex-ministr intérieur déjà examen infract dossi...,claud gué examen associ malfaiteur affair fina...
3,Pédocriminalité : quinze ans de réclusion requ...,L’homme de 70 ans comparait à huis clos pour r...,https://www.lemonde.fr/societe/article/2020/12...,2020,fr,ju,3,2,homm compar huis clos répondr accus viol agres...,pédocriminal réclus requ contr ex-chirurgien j...
4,Le prix littéraire Interallié décerné à Irène ...,"L’autrice, qui raconte les suites du meurtre d...",https://www.lemonde.fr/culture/article/2020/12...,2020,fr,cu,3,0,autric racont suit meurtr sœur aîn obtenu voix...,prix littérair interalli décern iren frain cri...


In [8]:
target_geo = ['afrique','asie', 'europe', 'france', 'am. latine', 'moyen-orient', 'espace', 'usa', 'monde']
target_topic = ['culture', 'economie', 'judiciaire', 'divers',  'politique', 'sciences', 'societe', 'sport']

__A. `Geo`__

In [14]:
pred_geo=best_geo_rf_entity_vocab.predict(sample_df.clean_text)
pred_geo_labels = [geo_code_dic[x] for x in pred_geo]
print(pred_geo_labels)

print(classification_report(sample_df.geo_code,  pred_geo))

['fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'fr', 'eu', 'fr', 'fr', 'fr', 'fr']
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         4
           3       0.36      1.00      0.53         4
           5       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1

    accuracy                           0.33        12
   macro avg       0.07      0.20      0.11        12
weighted avg       0.12      0.33      0.18        12



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 636 out of 636 | elapsed:    0.1s finished
/Users/eva/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


__B. `Topic`__

In [15]:
pred_topic = best_topic_lr_basic_vocab.predict(sample_df.clean_text)
pred_topic_labels = [topic_code_dic[x] for x in pred_topic]
print(pred_topic_labels)

print(classification_report(sample_df.topic_code,  pred_topic))

['eco', 'eco', 'eco', 'eco', 'eco', 'eco', 'eco', 'eco', 'eco', 'eco', 'eco', 'eco']
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       2.0
           1       0.00      0.00      0.00       0.0
           2       0.00      0.00      0.00       2.0
           4       0.00      0.00      0.00       5.0
           6       0.00      0.00      0.00       2.0
           7       0.00      0.00      0.00       1.0

    accuracy                           0.00      12.0
   macro avg       0.00      0.00      0.00      12.0
weighted avg       0.00      0.00      0.00      12.0



/Users/eva/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 2. Basic reco based on article body similarity



__A. Most similar article with tfidf vectorizer based on 726 articles__


In [16]:
## more occurate with geo codes ##

def most_similar(sample_articles, archive_articles, transformer=tfidf_vectorizer_base, refit = True):
    """
    Returns the index of the most similar articles
    sample_articles : dataframe, must have "text" col
    archive_articles : idem
    """
    if refit == True : 
        X = transformer.fit_transform(archive_articles.clean_text) # in case not already fitted, easier for testing
    else :
        X = transformer.transform(archive_articles.clean_text)
    sample_X = transformer.transform(sample_articles.clean_text)
    # compute similarity matrix
    sim_matrix = np.array([[cosine_similarity(sample_X[i], X[j]) for i in range(sample_X.shape[0])] for j in range(X.shape[0])])
    sim_matrix=sim_matrix.reshape([sim_matrix.shape[0], sim_matrix.shape[1]])
    reco_matrix = np.argmax(sim_matrix, axis=0)
    print(np.max(sim_matrix, axis=0))
    return reco_matrix



In [17]:
# tfidf_vectorizer_vocab
reco = most_similar(sample_df, labeled_df, transformer=tfidf_vectorizer_vocab)
for i in range(len(sample_df)):
    print("\n\n **", sample_df.title[i])
    print("\nMost similar article:", labeled_df.title[reco[i]])
    print(labeled_df.text[reco[i]][:300], '...')

[0.38578684 0.72876036 0.43501886 0.62348507 0.         0.
 0.5609347  0.42733268 0.60199348 0.82103965 0.43735419 0.50142543]


 ** « L’Ickabog », de J. K. Rowling : le triste royaume des Marcheurs

Most similar article: Les departements qui creent le plus d emplois suite de la page 122
LOIRE-ATLANTIQUE (44)
Nombre de Salariés au 31 décembre 1997 : 270 591
Taux de chômage au 31 mars 1998 : 14,5 %
Emplois créés en un an : 7 257 (+ 2,8 %)
Emplois créés depuis 1990 : 19 945 (+ 8 %)
La grande distribution, les services aux entreprises et le tourisme (hôtellerie-restauration) tirent les ...


 ** Covid-19 en France : Jean Castex expose la stratégie du gouvernement sur les vaccins

Most similar article: Cadres a temps partiel et fiers de l etre
Le mercredi, Catherine, gérante de placements dans une grande banque, fait de la musique et s'occupe de ses jeunes enfants. Christian, ingénieur chez Siemens-Nixdorf, ne travaille que le lundi, le mercredi et un vendredi sur deux. Le reste de la sema

In [18]:
# tfidf_vectorizer_base
reco = most_similar(sample_df, labeled_df, transformer=tfidf_vectorizer_base)
for i in range(len(sample_df)):
    print("\n\n **", sample_df.title[i])
    print("\nMost similar article:", labeled_df.title[reco[i]])
    print(labeled_df.text[reco[i]][:300], '...')

[0.27289489 0.37335101 0.43526006 0.48182959 0.31227106 0.30773593
 0.37628813 0.29000577 0.4629674  0.42372009 0.33879372 0.56163954]


 ** « L’Ickabog », de J. K. Rowling : le triste royaume des Marcheurs

Most similar article: Le Jurassic Park du communisme
A chaque instant, il se passe quelque chose au Grand Magasin n° 1... Au c?ur de Pyongyang, capitale de la Corée du Nord, le bâtiment ne désemplit pas. Il se passe quelque chose, certes. Mais quoi? Pas le moindre badaud dehors, sur le grand boulevard: les clients émergent de la bouche du métro et s'e ...


 ** Covid-19 en France : Jean Castex expose la stratégie du gouvernement sur les vaccins

Most similar article: L'affaire Boulin
Un corps à demi enfoncé dans l'eau noire des étangs de Hollande... Robert Boulin. Le suicide du plus ancien ministre du gouvernement prend aussitôt la dimension d'une affaire d'Etat. Albert du Roy explique la tragédie personnelle vécue par Boulin, et ses implications politiques. Elles vont bien au-d ..

__B. Random articles from the same cluster__

to add ==> save a kmeans model from nbk 02

In [33]:
X_LOC = tfidf_vectorizer_vocab_LOC.transform(labeled_df.clean_text)
X_sample_LOC = tfidf_vectorizer_vocab_LOC.transform(sample_df.clean_text)

pred_ref_clusters = kmeans_vocab_LOC.predict(X_LOC)
pred_sample_cluster = kmeans_vocab_LOC.predict(X_sample_LOC)

print(pred_sample_cluster)

for i in range(len(sample_df)):
    sample_article = sample_df[sample_df.index==i] # df
    print("\n\n **", sample_df.title[i])
    cluster_i = pred_sample_cluster[i]
    index = pred_ref_clusters==cluster_i
    #print(type(index))
    #print(index[:5])
    filtered_df_i = labeled_df[index]
    #print(len(filtered_df_i))
    #print(filtered_df_i[:5])
    #print(filtered_df_i.index)
    reco_i = choices(filtered_df_i.index, k=3)
    print("Recommandations: ")
    for r in reco_i:
        print(filtered_df_i[filtered_df_i.index==r].title.iloc[0])

[10  7 10  9 10 10  8 10  4  0 11  4]


 ** « L’Ickabog », de J. K. Rowling : le triste royaume des Marcheurs
Recommandations: 
Les troubles encheres du foot francais
Le ronflement
Un micro dans le ballon de foot


 ** Covid-19 en France : Jean Castex expose la stratégie du gouvernement sur les vaccins
Recommandations: 
Lvmh s offre les bijoux chaumet et les horloges ebel
CSG, ton univers impitoyable
1966: la crise avec les Américains


 ** Claude Guéant mis en examen pour « association de malfaiteurs » dans l’affaire du financement libyen de la campagne de 2007
Recommandations: 
Directrice trois jours par semaine
La victoire du c?ur
Revivre à Kangaroo Island


 ** Pédocriminalité : quinze ans de réclusion requis contre l’ex-chirurgien Joël Le Scouarnec
Recommandations: 
Bouygues bollore le climat se deteriore
Modes de vie: Temps libre
Pierre joxe invoque balladur pour contrer chirac


 ** Le prix littéraire Interallié décerné à Irène Frain pour « Un crime sans importance »
Recommandat

## 3. Similarity-based reco with filter on geo and topic


__A. Most similar article after filtering on same `geo`and `topic`__

still using the basic tfidf fitted on  726 articles

In [19]:
# note : not optimized, greedy filtering just for the sake of the demo
def filter_topic(sample_article, archives_articles):
    my_geo_code = sample_article.geo_code.iloc[0]
    my_topic_code = sample_article.topic_code.iloc[0]
    and_df = archives_articles[(archives_articles["geo_code"]==my_geo_code)&(archives_articles["topic_code"]==my_topic_code)].reset_index()
    if len(and_df)>0:
        ans = and_df
        print("# candidates:", len (ans), "(AND)")

    else :
        ans = archives_articles[(archives_articles["geo_code"]==my_geo_code)|(archives_articles["topic_code"]==my_topic_code)].reset_index()
        print("# candidates:", len (ans), "(OR)")
    return ans

In [20]:
for i in range(len(sample_df)):
    sample_article = sample_df[sample_df.index==i] # df
    print("\n\n **", sample_df.title[i])
    print(sample_article.geo_code.iloc[0])
    filtered_df = filter_topic(sample_article, labeled_df) # df with new index col
    reco = most_similar(sample_article, filtered_df, transformer=tfidf_vectorizer_base)
    print(reco)    
    print("\nMost similar article:", filtered_df[filtered_df.index==reco[0]].title.iloc[0])
    print(filtered_df.text[reco[0]][:300], '...')



 ** « L’Ickabog », de J. K. Rowling : le triste royaume des Marcheurs
2
# candidates: 8 (AND)
[0.18470781]
[0]

Most similar article: L'EUROPE SOUS INFLUENCE
De la Russie à l'Espagne, la cité royale, modèle politique de l'absolutisme, inspire les princes. Les architectes du roi exportent alors sa gloire. A la fin du xixe siècle, ce sont les imitations kitsch qui fleurissent.
Versailles n'est pas encore achevé. Mais la cause est déjà entendue. Le xviiie s ...


 ** Covid-19 en France : Jean Castex expose la stratégie du gouvernement sur les vaccins
3
# candidates: 73 (AND)
[0.35991131]
[1]

Most similar article: Un témoignage exclusif : ainsi finit Phnom Penh
Récit de l'entrée des Khmers rouges dans Phnom Penh, le 17 avril 1975
« La fête était finie. Tout craquait, tout s'effondrait. Il n'y avait plus de réjouissances, plus de lieux de plaisir. Les filles de joie avaient disparu, les bars étaient fermés. La nourriture avait atteint des prix astronomiques ...


 ** Claude Guéant mis en

__B. More elaborated criterias__